##### Solutia nr 3


La Solutia 3 am construit un clasificator multi-label antrenat pe weak labels din solutia 1 si solutia 2. Obiectivul a fost sa trec de la reguli si similaritate la un model care generalizeaza pe companii noi fara matching explicit. Am folosit embeddings minilm ca reprezentare stabila a textului. Am folosit logistic regression (ovr) pentru decizie pe fiecare eticheta. Am ales aceasta combinatie pentru balans intre performanta, timp de antrenare, explicabilitate si cost. Modelul invata direct din tiparele produse de reguli si de scorurile semantice. Modelul produce probabilitati pe etichete. Putem calibra praguri pe un dev set etichetat manual.

Am concatenat description, business tags si sector. Am curatat textul si am generat embeddings de 384 dimensiuni. Am consolidat etichetele din tfidf si embeddings intr un set final per companie. Am filtrat companiile fara etichete finale pentru training. Pentru setul curent, am avut ordinul miilor de companii si sute de etichete. Dimensionalitatea a ramas redusa si incarcarea pe processor a fost rezonabila.

Pe weak supervision nu exista ground truth complet. Regulile si similaritatea furnizeaza semnal suficient pentru un prim model. Acest semnal are bias dar ofera o baza uniforma. Clasificatorul invata frontiere de decizie si poate corecta partial erori locale din reguli. Putem imbunatati ulterior cu etichete umane. 


LR este un model liniar, explicabil. Coeficientii pe fiecare eticheta indica direct contributia dimensiunilor embedding. Antrenare rapida. Scalabilitate buna la sute de etichete. Output probabilistic. Usor de calibrat. Alternativele testate pe lista scurta au fost LinearSVC OVR si Random Forest. LinearSVC lipsit de probabilitati. Random Forest mai lent si mai greu de calibrat in multi-label cu vectori densi.

Weak labels contin bias. Daca solutia 1 si 2 propaga o eroare, modelul o invata. Clasele rare raman dificile. Dezechilibrul pe etichete impune praguri per clasa si weighting la antrenare. Descrieri scurte reduc semnalul. Multilingv necesita fie modele multilingve fie normalizare a limbii. Drift de business impune monitoring si reantrenare periodica.

D ce nu altceva. Fine tuning endto end cu transformere mari adauga cost si risc fara ground truth solid. Zero shot pur fara antrenare nu pastreaza consistent taxonomia interna. Clustering nesupravegheat nu asigura aliniere la etichete fixe. Am ales o ruta incrementala, auditala si reproductibila.

Ultima solotie 3, transforma semnalele din reguli si similaritate intr un model supervizat reproductibil. Castigam generalizare pe companii noi, timp mic de inferenta si control prin probabilitati. Modelul invata frontiere si reduce dependenta de matching literal. Coeficientii pe etichete permit explicatile. Calibrarea pragurilor per clasa ridica precizia in zone cu zgomot. Pentru seturi mari solutia ramane eficienta.

Solutia 3 ramane dependenta initiala de weak labels cu risc de propagare a bias ului si lipsa ground truth complet limiteaza masurarea absoluta a calitatii. Embeddings compacte nu ofera mapping direct la termeni, deci interpretarea nu arata cuvinte concrete.

Propun folosirea solutiei 1 pentru audit si reguli clare, solutia 2 pentru acoperire semantica si descoperire si solutia 3 pentru productie pentru ca serveste predictii rapide, explica deciziile cu coeficienti, ajusteaza praguri dupa obiectivele de business pe fiecare eticheta. In acest aranjament solutia 3 ofera echilibrul corect intre acuratete, viteza, cost si control.



In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

import warnings, os
warnings.filterwarnings("ignore")
os.environ["USE_TF"] = "0"
os.environ["TRANSFORMERS_NO_TF_WARNING"] = "1"


In [3]:

df = pd.read_csv('sol2_embeddings.csv')

print("Randuri:", len(df))
print("Coloane:", list(df.columns)[:10])
df.head(3)


Randuri: 9494
Coloane: ['description', 'business_tags', 'sector', 'category', 'niche', '_text', 'assigned_tf_idf', 'assigned_tf_idf_scores', 'assigned_emb', 'assigned_emb_scores']


,description,business_tags,sector,category,niche,_text,assigned_tf_idf,assigned_tf_idf_scores,assigned_emb,assigned_emb_scores
0,Welchcivils is a civil engineering and constru...,"['Construction Services', 'Multi-utilities', '...",Services,Civil Engineering Services,Other Heavy and Civil Engineering Construction,welchcivils is a civil engineering and constru...,[],{},['Commercial Communication Equipment Installat...,{'Commercial Communication Equipment Installat...
1,"Kyoto Vegetable Specialists Uekamo, also known...","['Wholesale', 'Dual-task Movement Products', '...",Manufacturing,Fruit & Vegetable - Markets & Stores,"Frozen Fruit, Juice, and Vegetable Manufacturing",kyoto vegetable specialists uekamo also known ...,[],{},"['Meat Processing Services', 'Food Processing ...",{'Meat Processing Services': 0.414367914199829...
2,Loidholdhof Integrative Hofgemeinschaft is a c...,"['Living Forms', 'Farm Cafe', 'Fresh Coffee', ...",Manufacturing,Farms & Agriculture Production,All Other Miscellaneous Crop Farming,loidholdhof integrative hofgemeinschaft is a c...,[],{},"['Gardening Services', 'Meat Processing Servic...","{'Gardening Services': 0.41804811358451843, 'M..."


In [4]:
import ast

def safe_eval(x):
    try:
        if isinstance(x, list):
            return x
        return ast.literal_eval(x)
    except:
        return []

df['assigned_tf_idf'] = df['assigned_tf_idf'].apply(safe_eval)
df['assigned_emb'] = df['assigned_emb'].apply(safe_eval)

# eticheta combinata
df['final_labels'] = df.apply(
    lambda r: list(set(r['assigned_tf_idf']) | set(r['assigned_emb'])),
    axis=1
)

df['final_labels_count'] = df['final_labels'].apply(len)
print("Companii cu etichete finale:", (df['final_labels_count'] > 0).sum())
df[['final_labels', 'final_labels_count']].head(5)


Companii cu etichete finale: 5296


,final_labels,final_labels_count
0,"[Commercial Construction Services, Residential...",8
1,"[Meat Processing Services, Food Processing Ser...",2
2,"[Cabinetry Manufacturing, Meat Processing Serv...",7
3,[Boiler Repair Services],1
4,[],0


In [5]:
# pastram doar companiile care au cel putin o eticheta finala
train_df = df[df['final_labels_count'] > 0].copy()
print("Companii pentru antrenare:", len(train_df))


Companii pentru antrenare: 5296


In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

texts = train_df['_text'].fillna('').tolist()
X_emb = model.encode(texts, show_progress_bar=False, batch_size=16, convert_to_numpy=True)
print("X_emb shape:", X_emb.shape)


X_emb shape: (5296, 384)


In [7]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(train_df['final_labels'])

print("Y shape:", Y.shape)
print("Etichete unice:", len(mlb.classes_))


Y shape: (5296, 220)
Etichete unice: 220


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_emb, Y, test_size=0.2, random_state=42)

print("Train:", X_train.shape, "Test:", X_test.shape)


Train: (4236, 384) Test: (1060, 384)


In [9]:
clf = OneVsRestClassifier(LogisticRegression(max_iter=1000, solver='saga'))
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("F1 macro:", f1_score(y_test, y_pred, average='macro'))
print("\nRaport pe clase:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=0))


F1 macro: 0.11447639011448889

Raport pe clase:
                                                  precision    recall  f1-score   support

                         Accessory Manufacturing       0.96      0.53      0.68        83
                            Advertising Services       1.00      0.06      0.11        17
                 Agricultural Equipment Services       1.00      0.32      0.48        60
             Agricultural Machinery Installation       0.00      0.00      0.00        13
                      Air Duct Cleaning Services       0.00      0.00      0.00         9
                     Alarm Installation Services       0.00      0.00      0.00         5
                        Animal Day Care Services       0.00      0.00      0.00        12
                       Animal Feed Manufacturing       1.00      0.36      0.53        14
                                 Animal Trainers       0.00      0.00      0.00         4
                        Animal Training Services   

In [10]:
import joblib
from pathlib import Path

output_dir = Path('s3')
output_dir.mkdir(exist_ok=True, parents=True)

joblib.dump(clf, output_dir / 'insurance_classifier.pkl')
joblib.dump(mlb, output_dir / 'mlb_labels.pkl')

print("Model și encoder salvate în folder:", output_dir)


Model și encoder salvate în folder: s3


In [12]:
# Un test pe posibile companii noi

new_companies = [
    "Digital platform for car insurance and claim management",
    "Veterinary clinic specialized in animal health and pet insurance",
    "Software company offering insurance risk analytics tools"
]

new_emb = model.encode(new_companies, batch_size=8, convert_to_numpy=True)
pred = clf.predict(new_emb)

pred_labels = mlb.inverse_transform(pred)

for txt, labs in zip(new_companies, pred_labels):
    print("\nText:", txt)
    print("Etichete prezise:", labs)



Text: Digital platform for car insurance and claim management
Etichete prezise: ('Insurance Services',)

Text: Veterinary clinic specialized in animal health and pet insurance
Etichete prezise: ('Animal Day Care Services', 'Animal Training Services', 'Veterinary Health Centers', 'Veterinary Services')

Text: Software company offering insurance risk analytics tools
Etichete prezise: ('Insurance Services',)


In [ ]:
# o versiune cu probabilitati

probas = clf.predict_proba(new_emb)
top_n = 3

for txt, row in zip(new_companies, probas):
    idx = np.argsort(row)[::-1][:top_n]
    labs = [mlb.classes_[i] for i in idx]
    scores = [round(row[i], 3) for i in idx]
    print("\nText:", txt)
    print("Top etichete:", list(zip(labs, scores)))



Text: Digital platform for car insurance and claim management
Top etichete: [('Insurance Services', 0.539), ('Digital Marketing Services', 0.123), ('Financial Services', 0.074)]

Text: Veterinary clinic specialized in animal health and pet insurance
Top etichete: [('Veterinary Services', 0.823), ('Animal Training Services', 0.781), ('Veterinary Health Centers', 0.74)]

Text: Software company offering insurance risk analytics tools
Top etichete: [('Insurance Services', 0.783), ('Risk Assessment Services', 0.181), ('Corporate Responsibility Services', 0.082)]
